In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings.ollama import OllamaEmbeddings

llm = ChatOllama(model="llama3:8b-instruct-q6_K")
embedding = OllamaEmbeddings(model="nomic-embed-text")

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://www.scholastic.com/hpread/HP_Book1_Chapter_Excerpt.pdf")
docs = loader.load()
len(docs)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10)
splits = text_splitter.split_documents(docs)
len(splits)

In [ ]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})
retriever_docs = retriever.invoke("Who is Harry Potter?")
retriever_docs

In [ ]:
# QA RAG Chain

from datetime import date

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.tools import tool


prompt_template = """Answer the question as precise as possible using the provided context. If the answer is not contained in the context, say "answer not available in context"
    Date: {date}
    Context: {context}
    Question: {question}
    Answer:
     """

prompt = PromptTemplate(
    template=prompt_template, input_variables=["date", "context", "question"]
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


@tool
def get_today(query: str) -> str:
    """Get today's date in the format YYYY-MM-DD."""
    today = date.today()
    return today.strftime("%Y-%m-%d")


chain = (
    {
        "date": get_today,
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

question = "What's the date today?"
retrieved_docs = retriever.invoke(question)
chain.invoke(question)

In [ ]:
get_today.description

In [ ]:
# QA RAG Chain

from datetime import date

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough


prompt_template = """Answer the question as precise as possible using the provided context. If the answer is not contained in the context, say "answer not available in context"
    Date: {date}
    Context: {context}
    Question: {question}
    Answer:
     """

prompt_template = PromptTemplate.from_template(
    """Answer the question as precise as possible using the provided context. If the answer is not contained in the context, say "answer not available in context"
    Date: {date}
    Context: {context}
    Question: {question}
    Answer:
     """
)

prompt_template.format(
    date=get_today(),
    context=format_docs(retriever_docs),
    question="Who is Harry Potter?",
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


def get_today():
    today = date.today()
    return today.strftime("%Y-%m-%d")


chain = (
    {
        "date": get_today,
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
question = "Tell me about Harry Potter"
retrieved_docs = retriever.invoke(question)
chain.invoke(question)